# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [ ]:
import sqlite3

In [ ]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [ ]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Again, we will use **`%%sql`** magic for our queries

In [ ]:
%load_ext sql
%sql sqlite:///wine.db

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [ ]:
%%sql DROP TABLE IF EXISTS wine;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1;
SELECT * FROM wine limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS producer;

-- Create producer table
CREATE TABLE producer AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1;
SELECT * FROM producer limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS production;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV, QTE, NP
FROM MASTER1
WHERE QTE is not NULL;
SELECT * FROM production limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS wine;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2;
SELECT * FROM wine limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS purchase;

-- Create purchase table
CREATE TABLE purchase AS
SELECT DISTINCT NV, DATES, LIEU, QTE, NB
FROM MASTER2
WHERE NV is not null and NB is not null;
SELECT * FROM purchase limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS client;

-- Create client table
CREATE TABLE client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2;
SELECT * FROM client limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS location;

-- Create location table
CREATE TABLE location AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2;
SELECT * FROM location limit 4;

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [ ]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT DISTINCT nb, NOM, PRENOM, TYPE as type_of_buveurs
FROM client
WHERE TYPE like 'gros' or TYPE like 'moyen';

In [ ]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [ ]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT *
FROM client 
WHERE client.NB NOT IN 
(SELECT purchase.NB
FROM purchase);

In [ ]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [ ]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT client.NB, client.NOM, client.PRENOM, client.TYPE
FROM purchase, client, wine
WHERE client.NB=purchase.NB and wine.NV=purchase.NV
GROUP BY purchase.NB;

In [ ]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [ ]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, sum(purchase.QTE) as QTE_BUE
FROM wine, purchase
WHERE wine.NV=purchase.NV and DATES LIKE "1983%"
GROUP BY purchase.LIEU, wine.CRU;

In [ ]:
# Test
%sql SELECT * FROM q83pl;

#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

In [ ]:
%%sql
DROP VIEW IF EXISTS q83pl1;
CREATE VIEW q83pl1 AS
SELECT LIEU, CRU, sum(purchase.QTE) as QTE_BUE
FROM wine, purchase
WHERE wine.NV=purchase.NV and DATES LIKE "1983%"
GROUP BY purchase.LIEU, wine.CRU
ORDER BY QTE ASC;
SELECT * FROM q83pl1;

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [ ]:
%%sql DROP TABLE IF EXISTS RBB;

-- Create RBB table
CREATE TABLE RBB AS
SELECT * FROM bons_buveurs
ORDER BY NB;

In [ ]:
# Test
%sql SELECT * FROM RBB;

#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [ ]:
%%sql
SELECT client.NB, client.NOM, client.PRENOM, client.TYPE, SUM(purchase.QTE) AS total
FROM client, purchase
WHERE client.NB=purchase.NB
GROUP BY client.NB, NOM, PRENOM
HAVING total>100;

Update instances

In [ ]:
%%sql
UPDATE client
SET TYPE = 'gros'
WHERE client.NB IN (SELECT client.NB
FROM client, purchase
WHERE client.NB=purchase.NB
GROUP BY client.NB, NOM, PRENOM
HAVING SUM(purchase.QTE)>100);

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [ ]:
%%sql
SELECT * FROM RBB ORDER BY NB;

In [ ]:
%%sql
SELECT * FROM bons_buveurs ORDER BY NB;

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [ ]:
%%sql
DROP TABLE IF EXISTS RBA;

CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec
ORDER BY NB;

In [ ]:
# Test
%sql SELECT * FROM RBA

#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [ ]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM, TYPE) VALUES (101,'Bouden','Sonia','petit');

In [ ]:
# Test
%sql SELECT * FROM RBA

#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [ ]:
# Test
%sql SELECT * FROM buveurs_asec;

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

In [ ]:
from IPython.display import Image
Image(filename='tp3.jpg')

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.